In [0]:
# Import necessary libraries

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, LongType, DateType, IntegerType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")


# Curated Zone 

read all necessary tables from curated database

In [0]:
# Read revenue agg table

revenue_agg_df = spark.read.table("curated_assignment.revenue_agg") \
                .withColumn("Month_Year", F.date_format(F.to_date(F.col("Month"), "MMM-yyyy"), "MM-yyyy"))

In [0]:
# Read customer agg table

customer_agg_df = spark.read.table("curated_assignment.customer_agg") \
                .withColumn("Month_Year", F.date_format(F.to_date(F.col("Month"), "MMM-yyyy"), "MM-yyyy"))

In [0]:
revenue_agg_df.cache()

display(revenue_agg_df.limit(5))

Status,Category,Payment_Method,Year,Month,City,County,Gender,Region,State,Customer_Segment,Revenue,Month_Year
complete,Mobiles & Tablets,cod,2021,May-2021,Tucson,Pima,M,West,AZ,middle-ages,1382.5,05-2021
received,Mobiles & Tablets,cod,2021,Jul-2021,Inland,Clay,F,Midwest,NE,adults,319.8,07-2021
complete,Mobiles & Tablets,cod,2021,Jun-2021,Elgin,Bastrop,F,South,TX,adults,999.9,06-2021
canceled,Mobiles & Tablets,bankalfalah,2021,Jun-2021,Memphis,Shelby,F,South,TN,old,4499.8,06-2021
received,Mobiles & Tablets,easypay_voucher,2021,Jun-2021,Phoenix,Maricopa,F,West,AZ,old,36.0,06-2021


In [0]:
revenue_agg_df.printSchema()

root
 |-- Status: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Payment_Method: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Customer_Segment: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Month_Year: string (nullable = true)




# Dashboard and Insights


### 1
Find revenue generated by different categories for the month of 11/2020.

In [0]:
# Assuming only completed orders lead to revenue

display(revenue_agg_df.where(F.col("Month")=="Nov-2020") \
                    .where(F.col("Status")=="complete") \
                    .groupby(["Category"]).agg(F.sum(F.col("Revenue")).alias("Total_Revenue")) \
                    .withColumn("Total_Revenue", F.round(F.col("Total_Revenue"),1)) \
                    .orderBy(F.col("Total_Revenue").desc())
        )

Category,Total_Revenue
Mobiles & Tablets,1372377.4
Entertainment,588682.6
Others,151800.0
Computing,143988.5
Appliances,124453.0
Women's Fashion,116139.9
Beauty & Grooming,66664.3
Men's Fashion,52660.0
Superstore,35032.1
Home & Living,20171.6


Databricks visualization. Run in Databricks to view.


## 2
Which top 5 categories have a maximum number of refunds in the year 2020?

In [0]:
# display(revenue_agg_df.select("Status").distinct())

In [0]:
display(revenue_agg_df.where(F.col("Year")=="2020") \
                    .where(F.col("Status")=="refund") \
                    .groupby(["Category"]).agg(F.count(F.col("Revenue")).alias("Refund_Count")) \
                    .orderBy(F.col("Refund_Count").desc()) \
                    .limit(5)
        )

Category,Refund_Count
Men's Fashion,258
Women's Fashion,104
Mobiles & Tablets,101
Appliances,87
Beauty & Grooming,70



## 3

Find a total number of orders by each category for each month and year in the dataset?

In [0]:
display(revenue_agg_df.head(4))

Status,Category,Payment_Method,Year,Month,City,County,Gender,Region,State,Customer_Segment,Revenue,Month_Year
complete,Mobiles & Tablets,cod,2021,May-2021,Tucson,Pima,M,West,AZ,middle-ages,1382.5,05-2021
received,Mobiles & Tablets,cod,2021,Jul-2021,Inland,Clay,F,Midwest,NE,adults,319.8,07-2021
complete,Mobiles & Tablets,cod,2021,Jun-2021,Elgin,Bastrop,F,South,TX,adults,999.9,06-2021
canceled,Mobiles & Tablets,bankalfalah,2021,Jun-2021,Memphis,Shelby,F,South,TN,old,4499.8,06-2021


In [0]:
# Ignoring the status and just getting total count of orders

display(revenue_agg_df.groupby(F.col("Category"),F.col("Month_Year"))
            .agg(F.count(F.col("Month_Year")).alias("Order_Count")) \
            .orderBy(["Category", "Month_Year"])
        )

Category,Month_Year,Order_Count
Appliances,01-2021,1301
Appliances,02-2021,682
Appliances,03-2021,1673
Appliances,04-2021,1887
Appliances,05-2021,761
Appliances,06-2021,2598
Appliances,07-2021,885
Appliances,08-2021,560
Appliances,09-2021,723
Appliances,10-2020,327


## 4

Find the total spend (in percentage of total spend of categories) by
 
customers by different age segments by different categories.

  E.g., Categories	Total Spend	Young	Adults	Middle-Age	Old
  
  E.g., In appliances category total spend is X amount of which a,b,c,d are percentages contributed from Young, Adults, Middle-Age and Old respectively. (a+b+c+d=100)

In [0]:
# Calculate total spend by age segment, ignoring status

revenue_cat_seg_agg_df = revenue_agg_df.groupBy("Category", "Customer_Segment") \
                                .agg(F.sum(F.col("Revenue")).alias("Total_Spend")) \
                                .withColumn("Total_Spend", F.round(F.col("Total_Spend"),0)) \
                                .withColumn('Total_Cat_Spend', F.sum(F.col("Total_Spend")).over(Window.partitionBy("Category"))) \
                                .withColumn("Segment_Percent_Category", F.round(((F.col("Total_Spend")*100)/F.col("Total_Cat_Spend")),1))


display(revenue_cat_seg_agg_df)               

Category,Customer_Segment,Total_Spend,Total_Cat_Spend,Segment_Percent_Category
Appliances,middle-ages,1.0431958E7,3.0060836E7,34.7
Appliances,old,9908120.0,3.0060836E7,33.0
Appliances,adults,8381944.0,3.0060836E7,27.9
Appliances,young,1338814.0,3.0060836E7,4.5
Beauty & Grooming,young,138837.0,2644747.0,5.2
Beauty & Grooming,middle-ages,827526.0,2644747.0,31.3
Beauty & Grooming,old,926114.0,2644747.0,35.0
Beauty & Grooming,adults,752270.0,2644747.0,28.4
Books,old,12523.0,32417.0,38.6
Books,middle-ages,10716.0,32417.0,33.1


In [0]:
# % spend by age segment

display(revenue_cat_seg_agg_df.groupBy("Category") \
                                .pivot("Customer_Segment") \
                                .sum("Segment_Percent_Category")) 

Category,adults,middle-ages,old,young
Appliances,27.9,34.7,33.0,4.5
Beauty & Grooming,28.4,31.3,35.0,5.2
Books,22.2,33.1,38.6,6.1
Computing,26.1,39.5,30.2,4.2
Entertainment,26.5,35.9,33.3,4.2
Health & Sports,20.4,39.2,37.2,3.2
Home & Living,25.9,32.5,36.4,5.3
Kids & Baby,25.9,31.4,35.4,7.2
Men's Fashion,25.3,35.2,33.3,6.3
Mobiles & Tablets,25.0,36.3,33.8,4.9


## 5

Spend by gender across different categories in terms of percentages.

In [0]:
revenue_cat_gen_agg_df = revenue_agg_df.groupBy("Category", "Gender") \
                           .agg(F.sum(F.col("Revenue")).alias("Total_Spend")) \
                           .withColumn("Total_Spend", F.round(F.col("Total_Spend"),0)) \
                           .withColumn('Total_Cat_Spend', F.sum(F.col("Total_Spend")).over(Window.partitionBy("Category"))) \
                          .withColumn("Gender_Percent_Category", F.round(((F.col("Total_Spend")*100)/F.col("Total_Cat_Spend")),1))

display(revenue_cat_gen_agg_df)

Category,Gender,Total_Spend,Total_Cat_Spend,Gender_Percent_Category
Appliances,M,1.5035771E7,3.0060836E7,50.0
Appliances,F,1.5025065E7,3.0060836E7,50.0
Beauty & Grooming,M,1371817.0,2644748.0,51.9
Beauty & Grooming,F,1272931.0,2644748.0,48.1
Books,M,19382.0,32417.0,59.8
Books,F,13035.0,32417.0,40.2
Computing,F,4305780.0,9362343.0,46.0
Computing,M,5056563.0,9362343.0,54.0
Entertainment,M,1.3693223E7,2.7138435E7,50.5
Entertainment,F,1.3445212E7,2.7138435E7,49.5


In [0]:
display(revenue_cat_gen_agg_df.groupBy("Category") \
                           .pivot("Gender") \
                           .sum("Gender_Percent_Category"))

Category,F,M
Appliances,50.0,50.0
Beauty & Grooming,48.1,51.9
Books,40.2,59.8
Computing,46.0,54.0
Entertainment,49.5,50.5
Health & Sports,40.8,59.2
Home & Living,54.8,45.2
Kids & Baby,49.6,50.4
Men's Fashion,47.2,52.8
Mobiles & Tablets,50.9,49.1


##6

Find top 5 customers for each month. The results should be as follows:   E.g., Year, Month, Custid, Name, country, Gender, Total Spend, Rank

In [0]:
display(customer_agg_df.head(5))

Cust_Id,Full_Name,Status,Category,Payment_Method,Year,Month,City,County,Gender,Region,State,Customer_Segment,Revenue,Month_Year
67734,"Renfroe, Frances",canceled,Appliances,Easypay,2020,Dec-2020,Bolton,Worcester,M,Northeast,MA,adults,195.6,12-2020
67989,"Owenby, Hallie",complete,Appliances,easypay_voucher,2020,Dec-2020,Page,Fayette,F,South,WV,middle-ages,0.0,12-2020
68690,"Wales, Coletta",received,Appliances,easypay_voucher,2020,Dec-2020,Ambler,Northwest Arctic,F,West,AK,middle-ages,40.41,12-2020
19934,"Terhune, Antonietta",complete,Appliances,cod,2020,Dec-2020,Pittsburgh,Allegheny,F,Northeast,PA,middle-ages,49.9,12-2020
70345,"Tarrant, Van",canceled,Appliances,jazzvoucher,2021,Jun-2021,Rhinehart,Catahoula,M,South,LA,young,75.757,06-2021


In [0]:
revenue_mon_cust_agg_df = customer_agg_df.groupBy("Month_Year", "Cust_Id", "Full_Name", "County", "Gender", "Revenue") \
                           .agg(F.sum(F.col("Revenue")).alias("Total_Spend")) \
                           .withColumn("Total_Spend", F.round(F.col("Total_Spend"),0)) \
                           .drop("Revenue")
display(revenue_mon_cust_agg_df.limit(5))

Month_Year,Cust_Id,Full_Name,County,Gender,Total_Spend
06-2021,107780,"Saulsbury, Sal",Harris,M,22.0
12-2020,68671,"Jacobson, Leisha",Alger,F,21969.0
10-2020,51717,"Vanzandt, Yi",Chittenden,F,2366.0
12-2020,72430,"Tinkham, Tristan",Clark,F,0.0
12-2020,77610,"Hostetter, Karole",Cumberland,F,0.0


In [0]:
windowSpec  = Window.partitionBy("Month_Year").orderBy(F.desc("Total_Spend"))

display(revenue_mon_cust_agg_df.withColumn("row_number",F.row_number().over(windowSpec)) \
                               .where(F.col("row_number")<6))

Month_Year,Cust_Id,Full_Name,County,Gender,Revenue,Total_Spend,row_number
05-2021,105009,"Lear, Elmo",Johnson,M,204469.80000000002,204470.0,1
05-2021,104056,"Duby, Eustolia",Vermilion,F,130177.49999999999,130177.0,2
05-2021,17987,"Davison, Jeff",Lawrence,M,101262.59,101263.0,3
05-2021,39707,"Beebe, Hortencia",Rutland,F,99198.6,99199.0,4
05-2021,12278,"Peake, Virgil",Robeson,M,84861.79999999999,84862.0,5
07-2021,110215,"Cobb, Joleen",Tishomingo,F,445469.80000000005,445470.0,1
07-2021,109213,"Hover, Margorie",Fergus,F,438480.0000000001,438480.0,2
07-2021,109038,"Dobbins, Everette",Winston,M,382225.89999999997,382226.0,3
07-2021,108786,"Overby, Diana",Morris,F,365123.79999999993,365124.0,4
07-2021,111127,"Champion, Tena",El Paso,F,336343.4,336343.0,5


##7

Calculate the RFM values for each customer (by customer id).

  RFM Represents R (Recency) F (Frequency) M (Monetary Value) (Click on the link to read more about RFM)
  For Recency calculation, use 31/10/2021 as last date. 
  So, the recency for any customer should be how many weeks before he or she has made the last purchase from the date of 01/10/2021.

In [0]:
# Recency (computing number of weeks since last purchase, making some rounding offs)
recency = customer_agg_df.groupBy("Cust_Id").agg(F.max("Month_Year").alias("max_date")) \
                   .withColumn("Max_Month_Day", F.to_date(F.concat(F.lit("1-"),F.lit(F.col("max_date"))),"dd-MM-yyyy")) \
                   .withColumn("Target_Date",F.to_date(F.lit("2021-10-31"), "yyyy-MM-dd")) \
                   .withColumn("recency",F.round(F.datediff(F.col("Target_Date"), F.col("Max_Month_Day"))/7,0)) \
                   .select("Cust_Id","recency")
                #    .withColumn("recency", F.when(10-F.col("Month_diff")>1, 10-F.col("Month_diff")).otherwise(1)) \
# Frequency
frequency = customer_agg_df.groupBy("Cust_Id").count().withColumnRenamed("count", "frequency")

# Monetary
monetary = customer_agg_df.groupBy("Cust_Id").agg(F.sum("Revenue").alias("monetary")) \
                        .withColumn("monetary",F.round(F.col("monetary"),0))

# Combine RFM
rfm_df = recency.join(frequency, "Cust_Id").join(monetary, "Cust_Id")


display(rfm_df)

Cust_Id,recency,frequency,monetary
89844,35.0,3,10365.0
87724,9.0,41,86217.0
89537,17.0,7,28703.0
8592,48.0,6,1125.0
7982,48.0,7,1463.0
96488,30.0,2,5050.0
18911,48.0,14,26210.0
74251,48.0,1,1691.0
27484,39.0,2,23427.0
109050,22.0,1,120.0


In [0]:
# To compute RFM value for a customer we are scaling 'recency', 'frequency','monetary' between 1-10

rfm_cols = ['recency', 'frequency','monetary']

for col_ in rfm_cols:
    max_val = rfm_df.select(F.max(col_)).collect()[0][0]
    min_val = rfm_df.select(F.min(col_)).collect()[0][0]

    diff_val = max_val - min_val
    
    col_name = col_ + "_norm"

    rfm_df = rfm_df.withColumn(col_name, F.round((((F.col(col_)-min_val)*9)/diff_val)+1,3))

    print(col_," | ",max_val, " | ",min_val)

recency  |  56.0  |  9.0
frequency  |  140  |  1
monetary  |  1900290.0  |  0.0


In [0]:
# We generate RFM as frequency*monetary/recency

display(rfm_df.withColumn("RFM", F.round(F.col("frequency_norm")*F.col("monetary_norm")/F.col("recency_norm"),3)))

Cust_Id,recency,frequency,monetary,recency_norm,frequency_norm,monetary_norm,RFM
4,48.0,21,27394.0,8.468,2.295,1.13,0.306
15,52.0,3,217.0,9.234,1.129,1.001,0.122
16,52.0,5,11869.0,9.234,1.259,1.056,0.144
20,9.0,7,28719.0,1.0,1.388,1.136,1.577
21,39.0,1,105.0,6.745,1.0,1.0,0.148
23,48.0,6,871.0,8.468,1.324,1.004,0.157
28,48.0,5,3971.0,8.468,1.259,1.019,0.152
32,48.0,108,114273.0,8.468,7.928,1.541,1.443
33,48.0,64,61875.0,8.468,5.079,1.293,0.776
41,52.0,1,220.0,9.234,1.0,1.001,0.108



## 8

Find top 10 customers based on frequency and monetary value. Sort them based on first frequency and then monetary value.

In [0]:
# Using the above dataframe computing the top 10 customers by frequency and monetary value

display(rfm_df.withColumn("RFM", 
                          F.round(F.col("frequency_norm")*F.col("monetary_norm")/F.col("recency_norm"),3)) \
        .orderBy(F.desc(F.col("RFM"))) \
        .limit(10) \
        .orderBy(F.desc("frequency_norm"),F.desc("monetary_norm"))
        )

Cust_Id,recency,frequency,monetary,recency_norm,frequency_norm,monetary_norm,RFM
83736,9.0,83,450105.0,1.0,6.309,3.132,19.76
83364,9.0,50,341216.0,1.0,4.173,2.616,10.917
94267,9.0,29,360583.0,1.0,2.813,2.708,7.618
109038,13.0,27,1900290.0,1.766,2.683,10.0,15.193
105009,9.0,23,534922.0,1.0,2.424,3.533,8.564
105971,9.0,18,640112.0,1.0,2.101,4.032,8.471
110215,9.0,14,1628440.0,1.0,1.842,8.712,16.048
105943,9.0,14,647793.0,1.0,1.842,4.068,7.493
111057,13.0,13,1368423.0,1.766,1.777,7.481,7.528
113694,9.0,8,1568178.0,1.0,1.453,8.427,12.244
